In [1]:
from joblib import Parallel, delayed
from utils import *

---

# 1. Loop

For-loop through VCF records and use functions from `utils.py` to extract annotation information

In [2]:
%%time

out_df = pd.DataFrame()  # initialize empty dataframe
vcf_read = vcf.Reader(open("test_vcf_data.txt", "r"))

for i, record in enumerate(vcf_read):
    if i < 5:  # only look through first few records for manageable output
        print(record, end = "\n\n")

        # unpack record into dictionary of depth 1
        record_unpacked = unpack_dict(record.__dict__)
        
        # unpack ALT allele objects
        record_unpacked = extract_alt(record_unpacked, alt_obj=record.ALT, alt_key="ALT")
            
        # unpack model._Call object
        call_dict = unpack_dict(
            extract_call(record_unpacked["samples"], format_str=record_unpacked["FORMAT"]),
        )
        # remove old keys
        del record_unpacked["samples"]
        del record_unpacked["FORMAT"]
        # merge dicts
        record_unpacked = {**record_unpacked, **call_dict}
    
        # get gene where mutation is
        overlap_dict = overlap_get(record)
        try:
            # first record
            record_unpacked["gene_id"] = overlap_dict[0]["gene_id"]
            record_unpacked["gene_symbol"] = overlap_dict[0]["external_name"]
            record_unpacked["gene_biotype"] = overlap_dict[0]["biotype"]
            record_unpacked["gene_description"] = overlap_dict[0]["description"]
            if len(overlap_dict) > 1:
                for i in range(1, len(overlap_dict)):
                    record_unpacked["gene_id{}".format(i)] = overlap_dict[i]["gene_id"]
                    record_unpacked["gene_symbol{}".format(i)] = overlap_dict[i]["external_name"]
                    record_unpacked["gene_biotype{}".format(i)] = overlap_dict[i]["biotype"]
                    record_unpacked["gene_description{}".format(i)] = overlap_dict[i]["description"]
        except:
            print("No gene overlap found for {}".format(record))

        # call Ensembl VEP endpoint
        vep_dict = vep_region_post(
            record,
            hgvs=1,
            #protein=1,
            #uniprot=1,
            GO=1,
            #vcf_string=1,
            #domains=1,
            distance=0,  # don't look for upstream/downstream effects for simplicity
        )
        # extract VEP info
        record_unpacked["most_severe_consequence"] = vep_dict["most_severe_consequence"]
        
        # get MAF and snp info if available
        if "colocated_variants" in vep_dict:
            for d in vep_dict["colocated_variants"]:
                if "minor_allele" in d:
                    record_unpacked["minor_allele"] = d["minor_allele"]
                    record_unpacked["minor_allele_freq"] = d["minor_allele_freq"]
                    record_unpacked["snp_id"] = d["id"]
        
        # add to out_df
        out_df = pd.concat([out_df, pd.DataFrame(record_unpacked, index=[i])])
    else:
        break  # after n loops, quit to look at output
        
calc_frequencies(out_df)  # calculate allele frequencies
    
print("\nDone! Total records: {}".format(i))

Record(CHROM=1, POS=1158631, REF=A, ALT=[G])

Record(CHROM=1, POS=1246004, REF=A, ALT=[G])

Record(CHROM=1, POS=1249187, REF=G, ALT=[A])

Record(CHROM=1, POS=1261824, REF=G, ALT=[C])

Record(CHROM=1, POS=1387667, REF=C, ALT=[G])


Done! Total records: 5
CPU times: user 178 ms, sys: 20.2 ms, total: 199 ms
Wall time: 10.4 s


In [3]:
out_df[["CHROM","POS","REF","ALT","ALT_type","NR","NV","NRef","VAF","RAF"]]

,CHROM,POS,REF,ALT,ALT_type,NR,NV,NRef,VAF,RAF
0,1,1158631,A,G,SNV,160,156,4,0.975000,0.025000
1,1,1246004,A,G,SNV,152,148,4,0.973684,0.026316
2,1,1249187,G,A,SNV,137,135,2,0.985401,0.014599
3,1,1261824,G,C,SNV,136,134,2,0.985294,0.014706
4,1,1387667,C,G,SNV,137,133,4,0.970803,0.029197


In [4]:
vep_dict["most_severe_consequence"]

'intron_variant'

In [5]:
[vep_dict["transcript_consequences"][x]["gene_symbol"] for x in range(len(vep_dict["transcript_consequences"]))]

['ATAD3C', 'ATAD3C']

In [6]:
[vep_dict["transcript_consequences"][x]["biotype"] for x in range(len(vep_dict["transcript_consequences"]))]

['protein_coding', 'protein_coding']

In [7]:
[vep_dict["transcript_consequences"][x]["consequence_terms"] for x in range(len(vep_dict["transcript_consequences"]))]

[['intron_variant'], ['intron_variant']]

In [15]:
vep_dict["transcript_consequences"]

[{'gene_symbol_source': 'HGNC',
  'gene_id': 'ENSG00000215915',
  'gene_symbol': 'ATAD3C',
  'strand': 1,
  'hgvsc': 'ENST00000378785.2:c.153-78C>G',
  'transcript_id': 'ENST00000378785',
  'consequence_terms': ['intron_variant'],
  'variant_allele': 'G',
  'hgnc_id': 32151,
  'go': 'GO:0000166:nucleotide_binding,GO:0005524:ATP_binding,GO:0006281:DNA_repair,GO:0006310:DNA_recombination,GO:0009378:four-way_junction_helicase_activity,GO:0017111:nucleoside-triphosphatase_activity',
  'impact': 'MODIFIER',
  'biotype': 'protein_coding'},
 {'variant_allele': 'G',
  'hgnc_id': 32151,
  'consequence_terms': ['intron_variant'],
  'flags': ['cds_end_NF'],
  'biotype': 'protein_coding',
  'impact': 'MODIFIER',
  'gene_symbol_source': 'HGNC',
  'transcript_id': 'ENST00000475091',
  'hgvsc': 'ENST00000475091.2:c.153-78C>G',
  'gene_id': 'ENSG00000215915',
  'gene_symbol': 'ATAD3C',
  'strand': 1}]

In [14]:
[i for i in range(len(vep_dict["transcript_consequences"])) if vep_dict["transcript_consequences"][i][""]]

[0, 1]

---

# 2. `joblib` Parallel

Try running in parallel to speed it up.

In [8]:
%%time
vcf_read = vcf.Reader(open("test_vcf_data.txt", "r"))
out = Parallel(n_jobs=5)(delayed(process_record)(record, i) for i, record in enumerate(vcf_read))

CPU times: user 29.2 s, sys: 1.47 s, total: 30.7 s
Wall time: 1h 4min 47s


In [11]:
len(out)  # list of pd.DataFrames returned

11765

In [12]:
out_df = pd.concat(out)  # concatenate into master df

In [13]:
out_df

,CHROM,POS,REF,ALT,QUAL,start,end,alleles,affected_start,affected_end,...,gene_biotype5,gene_description5,gene_id6,gene_symbol6,gene_biotype6,gene_description6,gene_id7,gene_symbol7,gene_biotype7,gene_description7
0,1,1158631,A,G,2965,1158630,1158631,A;G,1158630,1158631,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1246004,A,G,2965,1246003,1246004,A;G,1246003,1246004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1249187,G,A,2965,1249186,1249187,G;A,1249186,1249187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1261824,G,C,2965,1261823,1261824,G;C,1261823,1261824,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1387667,C,G,2965,1387666,1387667,C;G,1387666,1387667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11760,X,154020114,C,A,2965,154020113,154020114,C;A,154020113,154020114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11761,X,154456747,A,G,2965,154456746,154456747,A;G,154456746,154456747,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11762,X,155125435,AG,A,2973,155125434,155125436,AG;A,155125435,155125436,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11763,X,155127675,A,G,2965,155127674,155127675,A;G,155127674,155127675,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
out_df.columns

Index(['CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'start', 'end', 'alleles',
       'affected_start', 'affected_end', 'BRF', 'FR', 'HP', 'HapScore', 'MGOF',
       'MMLQ', 'MQ', 'NF', 'NR', 'PP', 'QD', 'SC', 'SbPval', 'Source', 'TC',
       'TCF', 'TCR', 'TR', 'WE', 'WS', 'sample', 'GT', 'GL', 'GOF', 'GQ', 'NV',
       'ALT_type', 'gene_id', 'gene_symbol', 'gene_biotype',
       'gene_description', 'most_severe_consequence', 'gene_id1',
       'gene_symbol1', 'gene_biotype1', 'gene_description1', 'FILTER',
       'gene_id2', 'gene_symbol2', 'gene_biotype2', 'gene_description2',
       'gene_id3', 'gene_symbol3', 'gene_biotype3', 'gene_description3',
       'gene_id4', 'gene_symbol4', 'gene_biotype4', 'gene_description4',
       'gene_id5', 'gene_symbol5', 'gene_biotype5', 'gene_description5',
       'gene_id6', 'gene_symbol6', 'gene_biotype6', 'gene_description6',
       'gene_id7', 'gene_symbol7', 'gene_biotype7', 'gene_description7'],
      dtype='object')

In [ ]:
# calculate VAF
out_df["VAF"] = out_df["NV"].astype(int) / out_df["NR"].astype(int)

# calculate reference reads
out_df["NWT"] = out_df["NR"].astype(int) - out_df["NV"].astype(int)

# calculate reference %
out_df["RAF"] = out_df["NWT"].astype(int) / out_df["NR"].astype(int)

In [21]:
out_df["ALT_type"].value_counts()

SNV    10838
MNV      891
Name: ALT_type, dtype: int64

In [22]:
out_df["most_severe_consequence"].value_counts()

intron_variant                         3942
missense_variant                       2549
synonymous_variant                     2243
non_coding_transcript_exon_variant     1184
splice_polypyrimidine_tract_variant     521
3_prime_UTR_variant                     299
splice_region_variant                   270
intergenic_variant                      262
5_prime_UTR_variant                     109
regulatory_region_variant                96
splice_donor_region_variant              85
frameshift_variant                       47
inframe_deletion                         25
inframe_insertion                        23
splice_donor_5th_base_variant            20
splice_acceptor_variant                  20
stop_gained                              17
splice_donor_variant                     14
stop_lost                                11
TF_binding_site_variant                  10
start_lost                                9
coding_sequence_variant                   7
stop_retained_variant           